In [ ]:
# ✅ STEP 1: Install required libraries (run in terminal if needed)
# pip install tensorflow opencv-python gradio scikit-learn matplotlib

# ✅ STEP 2: Import libraries
import os
import cv2
import numpy as np
import gradio as gr
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, BatchNormalization,
                                     Dropout, Dense, GlobalAveragePooling2D)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# ✅ STEP 3: Constants
IMG_SIZE = 128
BATCH_SIZE = 16
EPOCHS = 10
DATASET_PATH = './dataset'  # <-- Your local dataset folder path
CATEGORIES = ['yes', 'no']

# ✅ STEP 4: Load and preprocess data
data = []
labels = []

for category in CATEGORIES:
    folder_path = os.path.join(DATASET_PATH, category)
    print(f"Loading images for: {category}")
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        try:
            image = cv2.imread(img_path)
            if image is not None:
                image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
                data.append(image)
                labels.append(category)
        except Exception as e:
            print(f"Skipped {img_path}: {e}")

data = np.array(data) / 255.0

# Encode labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded, num_classes=2)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    data, labels_categorical, test_size=0.2, random_state=42)

# ✅ STEP 5: Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu',
           padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# ✅ STEP 6: Train model
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE,
                    validation_data=(X_test, y_test), epochs=EPOCHS, verbose=1)

# ✅ STEP 7: Prediction + Otsu thresholding + bounding box
def predict_with_bbox(image):
    image = image.resize((IMG_SIZE, IMG_SIZE))
    img_np = np.array(image)
    img_input = np.expand_dims(img_np / 255.0, axis=0)

    prediction = model.predict(img_input)
    pred_label = le.inverse_transform([np.argmax(prediction)])[0]

    output_img = img_np.copy()

    if pred_label == "yes":
        gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # Otsu's thresholding to separate bright tumor
        _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Morphological opening to remove noise
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        clean = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

        # Find contours
        contours, _ = cv2.findContours(clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Filter contours by area and solidity
        valid_contours = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area < 500:
                continue
            hull = cv2.convexHull(cnt)
            hull_area = cv2.contourArea(hull)
            if hull_area == 0:
                continue
            solidity = float(area) / hull_area
            if solidity > 0.5:
                valid_contours.append(cnt)

        if valid_contours:
            largest = max(valid_contours, key=cv2.contourArea)
            x, y, w, h = cv2.boundingRect(largest)
            cv2.rectangle(output_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    return pred_label, output_img

# ✅ STEP 8: Gradio Interface
interface = gr.Interface(
    fn=predict_with_bbox,
    inputs=gr.Image(type='pil'),
    outputs=[
        gr.Label(label="Prediction"),
        gr.Image(label="Tumor Highlighted Image (if any)")
    ],
    title="Brain Tumor Detector with Otsu Thresholding",
    description="CNN classifies MRI as Yes/No. If Yes, Otsu's thresholding highlights tumor area with bounding box."
)

# ✅ STEP 9: Launch app
if __name__ == "__main__":
    interface.launch()
